In [0]:
import pandas as pd
import numpy as np

In [0]:
df_bt=spark.table("default.sw_indices_merged").toPandas() #Spark DF to Pandas DF
df_bt["Date"]=pd.to_datetime(df_bt['Date']) #Convert to datetime
df_bt["Date"]=df_bt["Date"].dt.date
df_bt=df_bt.sort_values("Date").reset_index(drop=True) #Sort values based on Date ; Convert index to column
df_bt.head(5)

In [0]:
df_bt["SMA_50"]=df_bt["SwarajEng_Close"].rolling(window=50).mean().round(2) #SMA50
df_bt["SMA_200"]=df_bt["SwarajEng_Close"].rolling(window=200).mean().round(2) #SMA200
df_bt["signal"]=0 #Signal 0 by default
df_bt.loc[(df_bt["SMA_50"]>df_bt["SMA_200"]) & (df_bt["SMA_50"].shift(1)<df_bt["SMA_200"].shift(1)),"signal"]=1 #If crosses SMA200 then 1
df_bt["position"]=np.where(df_bt["SMA_50"]>df_bt["SMA_200"],1,0)

df_bt["daily_return"]=df_bt["SwarajEng_Close"].pct_change()
df_bt["cum_daily_return"]=(df_bt["daily_return"]+1).cumprod()
df_bt["cum_daily_return"]=(df_bt["cum_daily_return"]-1)*100

#Days when we entered the trade/investment

df_bt["strategy_return_sma"]=df_bt["daily_return"]*df_bt["position"]
df_bt["cum_strategy_return_sma"]=(df_bt["strategy_return_sma"]+1).cumprod()
df_bt["cum_strategy_return_sma"]=((df_bt["cum_strategy_return_sma"]-1)*100).round(2)

#Days when we didn't even enter

df_bt["missed_return_sma"]=np.where(df_bt["position"]==0,df_bt["daily_return"],0)
df_bt["cum_missed_return_sma"]=(df_bt["missed_return_sma"]+1).cumprod()
df_bt["cum_missed_return_sma"]=((df_bt["cum_missed_return_sma"]-1)*100).round(2)

In [0]:
# Detect Max_Drawdown
df_bt["peak"]=df_bt["SwarajEng_Close"].cummax()
df_bt["drawdown"] = (df_bt["SwarajEng_Close"] - df_bt["peak"]) / df_bt["peak"]
max_drawdown = df_bt["drawdown"].min().round(4) * 100
print(f"Max drawdown is {max_drawdown:.2f}%")


In [0]:
#CAGR
start_value=df_bt["SwarajEng_Close"].iloc[0]
end_value=df_bt["SwarajEng_Close"].iloc[-1] 
n_days=(df_bt["Date"].iloc[-1]-df_bt["Date"].iloc[0]).days #No: of days
n_years=n_days/365.25
CAGR=(((end_value/start_value)**(1/n_years))-1)*100  #Typically we have just for 2 years
print(f"CAGR for the Swaraj in last 2 years is: {CAGR:.2f} %")

In [0]:
#Calmar's Ratio 
Calmar_ratio=CAGR/abs(max_drawdown)
print(f"Ratio for the Swaraj {Calmar_ratio:.2f}")

In [0]:
#Bollinger Band
df_bt["middle_band"]=df_bt["SwarajEng_Close"].rolling(window=20).mean()
df_bt["upper_band"]=df_bt["middle_band"]+(2*df_bt["SwarajEng_Close"].rolling(window=20).std())
df_bt["lower_band"]=df_bt["middle_band"]-(2*df_bt["SwarajEng_Close"].rolling(window=20).std())

df_bt["signal_bb"]=0
df_bt.loc[(df_bt["SwarajEng_Close"]>df_bt["lower_band"]) & (df_bt["SwarajEng_Close"].shift(1)<=df_bt["upper_band"].shift(1)),"signal_bb"]=1

df_bt["position_bb"]=np.where(df_bt["SwarajEng_Close"]>df_bt["lower_band"],1,0)

df_bt["strategy_returns_bb"]=df_bt["daily_return"]*df_bt["position_bb"]
df_bt["cum_strategy_returns_bb"]=(1+df_bt["strategy_returns_bb"]).cumprod()
df_bt["cum_strategy_returns_bb"] = ((df_bt["cum_strategy_returns_bb"] - 1) * 100).round(2)
df_bt["missed_return_bb"] = np.where(df_bt["position_bb"] == 0, df_bt["daily_return"], 0)
df_bt["cum_missed_return_bb"] = (1 + df_bt["missed_return_bb"]).cumprod()
df_bt["cum_missed_return_bb"] = ((df_bt["cum_missed_return_bb"] - 1) * 100).round(2)

In [0]:
from pyspark.sql import functions as F
spark_df=spark.createDataFrame(df_bt)
spark_df.write.option("overwriteSchema","true").mode("overwrite").format("delta").saveAsTable("default.sw_strategy")

In [0]:
%sql
select *
from sw_strategy
order by `Date` desc
limit 500